In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-t2m_1979-2005_NDJ_1D_histday.nc')
rhData  = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-rh2m_1979-2005_NDJ_1D_histday.nc')
u10Data = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-u10m_1979-2005_NDJ_1D_histday.nc')
v10Data = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-V10M_1979-2005_NDJ_1D_histday.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-t850_1979-2005_NDJ_1D_histday.nc')
zLevData = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-z850_1979-2005_NDJ_1D_histday.nc')
wLevData = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-w700_1979-2005_NDJ_1D_histday.nc')
uLevData = xr.open_dataset('../../../Data/LMDZ_Data/RegridLMDZOR-HIST-u850_1979-2005_NDJ_1D_histday.nc')

In [6]:
t2m1=t2mData.t2m.rename({'time_counter':'time'})
datetimeindex = t2m1.indexes['time'].to_datetimeindex()
t2m1['time'] = datetimeindex

u101=u10Data.u10m.rename({'time_counter':'time'})
datetimeindex = u101.indexes['time'].to_datetimeindex()
u101['time'] = datetimeindex

v101=v10Data.v10m.rename({'time_counter':'time'})
datetimeindex = v101.indexes['time'].to_datetimeindex()
v101['time'] = datetimeindex

rh1=rhData.rh2m.rename({'time_counter':'time'})
datetimeindex = rh1.indexes['time'].to_datetimeindex()
rh1['time'] = datetimeindex

tLev1=tLevData.t850.rename({'time_counter':'time'})
datetimeindex = tLev1.indexes['time'].to_datetimeindex()
tLev1['time'] = datetimeindex

uLev1=uLevData.u850.rename({'time_counter':'time'})
datetimeindex = uLev1.indexes['time'].to_datetimeindex()
uLev1['time'] = datetimeindex

w7001=wLevData.w700.rename({'time_counter':'time'})
datetimeindex = w7001.indexes['time'].to_datetimeindex()
w7001['time'] = datetimeindex

zLev1=zLevData.z850.rename({'time_counter':'time'})
datetimeindex = zLev1.indexes['time'].to_datetimeindex()
zLev1['time'] = datetimeindex



<ipython-input-6-66baf6541250>:2: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t2m1.indexes['time'].to_datetimeindex()
<ipython-input-6-66baf6541250>:6: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = u101.indexes['time'].to_datetimeindex()
<ipython-input-6-66baf6541250>:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da

In [7]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v101.values**2)+(u101.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v101.time,'latitude': v101.latitude,'longitude': v101.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rh1/100.0)},
                   coords={'time': v101.time,'latitude': v101.latitude,'longitude': v101.longitude})

#Calculate inv
inv=t2m1.values-tLev1.values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v101.time,'latitude': v101.latitude,'longitude': v101.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLev1.values-u101.values)/(zLev1.values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v101.time,'latitude': v101.latitude,'longitude': v101.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [8]:
# AO data
AOData = xr.open_dataset('LMDZ-AOindex-NDJF-Daily-1980-2014.nc')
aoTS=AOData.AO

Darray=np.zeros((AOData.time.shape[0],t2m1.latitude.shape[0], t2m1.longitude.shape[0]))
for t in range(aoTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2m1.latitude.shape[0], t2m1.longitude.shape[0]), aoTS[t].values)
AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2m1.latitude,'longitude': t2m1.longitude}) 
# EU data
EUData = xr.open_dataset('LMDZ-EUindex-NDJF-Daily-1980-2014.nc')
EUData.EU
euTS=EUData.EU

Darray=np.zeros((EUData.time.shape[0],t2m1.latitude.shape[0], t2m1.longitude.shape[0]))
for t in range(euTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2m1.latitude.shape[0], t2m1.longitude.shape[0]), euTS[t].values)
EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2m1.latitude,'longitude': t2m1.longitude})

In [9]:
# create mask
oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [10]:
#AO
AO5D=AOData.AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

#EU
EU5D=EUData.EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

In [11]:
t1=AO5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
AO5DAll.values=t1.unstack()

t1=EU5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
EU5DAll.values=t1.unstack()

AO5DAll.values=AO5DAll.values*mask
AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

EU5DAll.values=EU5DAll.values*mask
EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)

In [14]:
t2m=t2m1.shift(time=1)
ws=ws_ds.ws.shift(time=1)
rh=rh_ds.rh.shift(time=1)
inv=inv_ds.inv.shift(time=1)
w=w7001.shift(time=1)
ushear=ushear_ds.ushear.shift(time=1)

In [15]:
t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))
rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))
invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))
ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2020)].sel(time=slice('1980-1-1','2005-12-31'),latitude=slice(35,0),longitude=slice(50,100))

In [16]:
t1=t2mTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
t2mTsAll.values=t1.unstack()

t1=wsTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
wsTsAll.values=t1.unstack()

t1=rhTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
rhTsAll.values=t1.unstack()

t1=invTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
invTsAll.values=t1.unstack()

t1=ushearTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
ushearTsAll.values=t1.unstack()


t1=wTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
wTsAll.values=t1.unstack()

/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr

In [17]:
t2mTsAll.values=t2mTsAll.values*mask
wsTsAll.values=wsTsAll.values*mask
rhTsAll.values=rhTsAll.values*mask
invTsAll.values=invTsAll.values*mask
ushearTsAll.values=ushearTsAll.values*mask
wTsAll.values=wTsAll.values*mask


In [18]:
t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)


In [19]:
t2mt=t2mTsAll.values
t2mt=t2mt[:,:,:,None]
t2mt.shape


wst=wsTsAll.values
wst=wst[:,:,:,None]
wst.shape

rht=rhTsAll.values
rht=rht[:,:,:,None]
rht.shape


invt=invTsAll.values
invt=invt[:,:,:,None]
invt.shape

wt=wTsAll.values
wt=wt[:,:,:,None]
wt.shape

usheart=ushearTsAll.values
usheart=usheart[:,:,:,None]
usheart.shape

aot=AO5DAll.values
aot=aot[:,:,:,None]
aot.shape

eut=EU5DAll.values
eut=eut[:,:,:,None]
eut.shape


(1612, 18, 26, 1)

In [20]:
X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
X.shape

(8, 1612, 18, 26, 1)

In [21]:
X_reshape = np.einsum('lkija->klija',X)
X_reshape.shape

(1612, 8, 18, 26, 1)

In [22]:
# Load saved model
# load model
model = load_model('../../March2021/Observation_models/modelCNN.h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 8, 18, 26, 16)     448       
_________________________________________________________________
average_pooling3d (AveragePo (None, 4, 9, 13, 16)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 4, 9, 13, 32)      13856     
_________________________________________________________________
average_pooling3d_1 (Average (None, 2, 5, 7, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 5, 7, 32)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 2, 5, 7, 64)       55360     
_________________________________________________________________
average_pooling3d_2 (Average (None, 1, 3, 4, 64)       0

In [23]:
yLR=model.predict(X_reshape)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})

In [24]:
dump(y_predLin_ds.yLR,'../Model_plots/LMDZ-CNN-Y.joblib')

['../Model_plots/LMDZ-CNN-Y.joblib']